In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd

In [2]:
revenue_url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
ebay_url = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=Play+station+5&_sacat=0&LH_TitleDesc=0&_odkw=ps+5&_osacat=0"

In [3]:
client = urlopen(revenue_url)
client_ebay = urlopen(ebay_url)

In [4]:
page_html = client.read()
page_eb_html = client_ebay.read()
client_ebay.close()
client.close() # close connection

In [5]:
client.read()

b''

In [6]:
page_soup = soup(page_html)
page_ebay_soup = soup(page_eb_html)

In [7]:
page_ebay_soup.div

<div class="x-header"><script type="text/javascript">var GHpre = {si:false}</script><style>.gh-hide-if-nocss{display:none;}.gh-ar-hdn{color:#fff}</style> <div class="gh-acc-exp-div gh-hide-if-nocss"><a class="gh-acc-a" href="#mainContent" id="gh-hdn-stm">Skip to main content</a></div><!--[if lt IE 9]><div id="gh" role="banner" class="gh-IE8 gh-flex gh-pre-js gh-w "><![endif]--><header class="gh-ui-6-5 gh-flex gh-pre-js gh-w gh-sch-prom" data-treatment="" id="gh" role="banner"><table class="gh-tbl" role="presentation"><tbody><tr><td class="gh-td"><a _sp="m570.l2586" href="https://www.ebay.com/" id="gh-la"><img alt="eBay Home" height="200" id="gh-logo" src="https://ir.ebaystatic.com/rs/v/fxxj3ttftm5ltcqnto1o4baovyl.png" style="clip:rect(47px, 118px, 95px, 0px); position:absolute; top:-47px;left:0" width="250"/></a></td><td class="gh-td"><div class="gh-hide-if-nocss" id="gh-shop"><button aria-controls="gh-sbc-o" aria-expanded="false" class="gh-control" id="gh-shop-a"> Shop by category<i c

In [8]:
items_to_extract = {"s-item__title": "div", "s-item__price": "span", "s-item__location s-item__itemLocation": "span"}
cols = [i.split("__")[-1].capitalize() for i in list(items_to_extract.keys())]
cols

['Title', 'Price', 'Itemlocation']

In [9]:
item_info = page_ebay_soup.findAll("div", class_ = "s-item__info")
item_info[0].select("div.s-item__title, span.s-item__price, span.s-item__location")

[<div class="s-item__title"><span aria-level="3" role="heading"><!--F#f_0-->Shop on eBay<!--F/--></span></div>,
 <span class="s-item__price"><!--F#f_0--><!--F#f_0-->$20.00<!--F/--><!--F/--></span>]

In [21]:
from collections import defaultdict

items_list = defaultdict(list)

def get_pages(page):
    page_items = page.select("div.s-item__title, span.s-item__price, span.s-item__location")
    if len(page_items) == len(items_to_extract):
        for item in page_items:
            if item.get("class")[0] == "s-item__title":
                items_list[item.get("class")[0]].append(item.span.text)
            else:
                items_list[item.get("class")[0]].append(item.text.strip("$"))
    

# all_pages = page_ebay_soup.select("div.s-item__title, span.s-item__price, span.s-item__location")


# items_list = defaultdict(list)


# for item in all_pages:
#     if item.get("class")[0] == "s-item__title":
#         items_list[item.get("class")[0]].append(item.span.text)
#     else:
#         items_list[item.get("class")[0]].append(item.text.strip("$"))
list(map(get_pages, item_info)) #
items_list['s-item__price'] = pd.to_numeric(items_list['s-item__price'], errors='coerce')
df = pd.DataFrame(dict(items_list))
df.columns = cols
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("ps5.csv", index=False)

In [23]:
# cls = {"class": None}
# dd = dict.fromkeys(items_to_extract.keys(), None)

# #standard
# # page_soup.find("table", {"class": "wikitable sortable"})

# # Much easier way
# #page_soup.find("table", class_ = "wikitable sortable")
# # cls['class'].append("s-item_price")
# for item, val in items_to_extract.items():
#     cls["class"] = item
#     dd[item] = page_ebay_soup.findAll(val, cls)
# dd

In [24]:
# from collections import defaultdict
# item_list = {}

# def strip_span(i, t_type):
#     return i.span.text if "title" in t_type else i.text.strip("$")

# # all_d = {"title": [], "price": []}
# price_list = []
# for item in ded:
#     print(item)
#     item_list[item] = list(map(lambda x: strip_span(x, item), ded[item]))

In [24]:
# items_list['s-item__price']

In [16]:
# items_list['s-item__price'] = pd.to_numeric(items_list['s-item__price'], errors='coerce')

In [23]:
# newitem_list = list(items_list.values())
# newitem_list

In [22]:
# df = pd.DataFrame(list(zip(*newitem_list)), columns=cols)
# df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_csv("ps5.csv", index=False)

In [29]:
df1 = pd.read_csv("ps5.csv")
df1

,Title,Price,Itemlocation
0,Shop on eBay,20.00,from Japan
1,PS5 Sony PlayStation 5 Console Disc Version!,449.99,from United States
2,New ListingPS5 Sony PlayStation 5 Console Disc...,350.00,from United States
3,PlayStation 5 Digital Console Cover Marvels Sp...,174.99,from United States
4,PlayStation5 Console Horizon Forbidden West Bu...,499.99,from United States
5,Sony PS5 Blu-Ray Edition Console Spider-Man 2 ...,699.99,from United States
6,SONY PS5 - PLAYSTATION 5 - SYSTEM - DISC - CFI...,375.00,from United States
7,PlayStation 5 Digital Edition God of War Ragna...,459.00,from United States
8,Sony PlayStation 5 825GB Digital Edition Home ...,375.00,from United States
9,Sony Playstation 5 Disc Version Console with D...,507.99,from United States


In [30]:
# Scrape tables from websites\



#table already extracted and stored in variable called page_soup    


In [25]:
# page_soup

In [32]:
table = page_soup.find("table", {"class": "wikitable sortable"})
table

<table class="wikitable sortable">
<caption>
</caption>
<tbody><tr>
<th>Rank
</th>
<th>Name
</th>
<th>Industry
</th>
<th>Revenue <br/>(USD millions)
</th>
<th>Revenue growth
</th>
<th>Employees
</th>
<th>Headquarters
</th></tr>
<tr>
<td>1
</td>
<td><a href="/wiki/Walmart" title="Walmart">Walmart</a>
</td>
<td><a href="/wiki/Retail" title="Retail">Retail</a>
</td>
<td style="text-align:center;">611,289
</td>
<td style="text-align:center;"><span typeof="mw:File"><span title="Increase"><img alt="Increase" class="mw-file-element" data-file-height="300" data-file-width="300" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/11px-Increase2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/17px-Increase2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/22px-Increase2.svg.png 2x" width="11"/></span></span> <span data-sort-value="7000300000000000000♠" style="display:none"></span> 6.

In [33]:
cols = [x.text.strip() for x in table.findAll("th")]
cols

['Rank',
 'Name',
 'Industry',
 'Revenue (USD millions)',
 'Revenue growth',
 'Employees',
 'Headquarters']

In [34]:
companies = pd.DataFrame(columns=cols)
companies

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


In [35]:
for row in table.findAll("tr")[1:]:
    rows_data = row.findAll("td")
    extracted_row = [data.text.strip() for data in rows_data]
    idx = len(companies)
    companies.loc[idx] = extracted_row
companies.to_csv("large_companies.csv", index=False)

In [36]:
comp2 = pd.read_csv("large_companies.csv")
comp2

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"611,289",6.7%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"513,983",9.4%,"1,540,000","Seattle, Washington"
2,3,Exxon Mobil,Petroleum industry,"413,680",44.8%,"62,000","Spring, Texas"
3,4,Apple,Electronics industry,"394,328",7.8%,"164,000","Cupertino, California"
4,5,UnitedHealth Group,Healthcare,"324,162",12.7%,"400,000","Minnetonka, Minnesota"
...,...,...,...,...,...,...,...
95,96,Best Buy,Retail,"46,298",10.6%,"71,100","Richfield, Minnesota"
96,97,Bristol-Myers Squibb,Pharmaceutical industry,"46,159",0.5%,"34,300","New York City, New York"
97,98,United Airlines,Airline,"44,955",82.5%,"92,795","Chicago, Illinois"
98,99,Thermo Fisher Scientific,Laboratory instruments,"44,915",14.5%,"130,000","Waltham, Massachusetts"
